In [58]:
%load_ext autoreload
%autoreload 2
PROJ_HOME='/Users/keemsunguk/Projects/analyst_performance/'
import sys
sys.path.append(PROJ_HOME)
import gc
from datetime import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import copy
import pickle
from tqdm import tqdm

In [3]:
from pathlib import Path

In [4]:
data_dir = Path(PROJ_HOME+'/data')

In [5]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_lg")

In [7]:
from googletrans import Translator

# Load input data

In [61]:
merged_df = pd.read_excel(data_dir/'merged_translated_fix2.xls')

# Translate

In [9]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.kr',
    ])

In [62]:
merged_df

,Unnamed: 0,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,...,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title
0,0,백영찬,KB,A078930,GS,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,...,41300.0,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,51400,51400,4분기 실적은 기대수준을 소폭 하회할 전망
1,1,백영찬,KB,A010060,OCI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,폴리실리콘 원가개선 필요한 시점,62700,...,52200.0,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,65900,67300,폴리실리콘 원가개선 필요한 시점
2,2,백영찬,KB,A011780,금호석유,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,...,66100.0,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,82100,82100,"견조한 고무사업, 그러나 부진한 페놀유도체"
3,3,장정훈,삼성,A006400,삼성SDI,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,20년 연초에 꾸는 꿈,237000,...,324500.0,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,322500,351000,20년 연초에 꾸는 꿈
4,4,백영찬,KB,A051910,LG화학,2020-01-02 00:00:00,2019-12-30 00:00:00,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,...,412000.0,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,391500,422500,4분기 ESS 비용반영을 긍정적으로 보자
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,39816,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,182500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000",호실적과 편안한 2022년
39796,39817,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,10850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950",공자위의 조속한 매각 결단을 환영
39797,39818,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,124500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500",저가 매수 유효
39798,39819,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지",8840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880","숨은 진주, 3Q 호실적 & 신재생에너지"


In [63]:
#merged_df.reset_index(inplace=True)
merged_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [12]:
eng_title = []
for k, r in tqdm(merged_df.iterrows(), total=39892):
    try:
        ex1 = translator.translate(r['report_title'] , src="ko", dest="en")
        eng_title.append(ex1.text)
    except:
        print(k, r['report_title'])
        eng_title.append(r['report_title'])

 48%|████▊     | 19281/39892 [6:41:30<5636:31:26, 984.50s/it]

19280 NIM 방어, 비이자 회복세로 컨센서스 상회


 48%|████▊     | 19324/39892 [8:42:18<10206:31:23, 1786.44s/it]

19323 3Q 크게 상회. 분기 이익 1조원으로 레벨 업


 49%|████▊     | 19367/39892 [10:32:30<11222:01:47, 1968.30s/it]

19366 MLCC 이익 사이클


100%|█████████▉| 39800/39892 [16:40:16<02:18,  1.51s/it]        


In [13]:
merged_df['en_report_title'] = eng_title

In [18]:
merged_df

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title
0,백영찬,KB,A078930,GS,43832,43829,NaN,4분기 실적은 기대수준을 소폭 하회할 전망,51300,50000,...,41300.0,41000.0,39400.0,40450.0,40450.0,40550.0,36400.0,51400,51400,4Q09 earnings are expected to meet expectations
1,백영찬,KB,A010060,OCI,43832,43829,NaN,폴리실리콘 원가개선 필요한 시점,62700,62800,...,52200.0,52000.0,50100.0,48450.0,48200.0,46300.0,39150.0,65900,67300,Polysilicon cost improvement required
2,백영찬,KB,A011780,금호석유,43832,43829,NaN,"견조한 고무사업, 그러나 부진한 페놀유도체",77300,76000,...,66100.0,65900.0,64200.0,62800.0,63200.0,63700.0,56300.0,82100,82100,"A solid rubber business, but sluggish phenol oil"
3,장정훈,삼성,A006400,삼성SDI,43832,43829,NaN,20년 연초에 꾸는 꿈,237000,232000,...,324500.0,323500.0,305500.0,298000.0,301500.0,291500.0,284500.0,322500,351000,Dreams of 20 years
4,백영찬,KB,A051910,LG화학,43832,43829,NaN,4분기 ESS 비용반영을 긍정적으로 보자,321000,314000,...,412000.0,404000.0,385500.0,380000.0,377500.0,367000.0,343500.0,391500,422500,Let's see a positive reflection of ESS costs i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39795,김록호,하나금융,A009150,삼성전기,9/10/21,9/9/21,9/8/21,호실적과 편안한 2022년,"182,500","188,000",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"188,000","188,000",Room and comfortable 2022
39796,최정욱,하나금융,A316140,우리금융지주,9/10/21,9/9/21,9/8/21,공자위의 조속한 매각 결단을 환영,"10,850","10,950",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"10,950","10,950",Welcome to the confusion of the confidence
39797,심은주,하나금융,A271560,오리온,9/10/21,9/9/21,9/8/21,저가 매수 유효,"124,500","123,500",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"124,500","124,500",Low cost volume valid
39798,박강호,대신,A229640,LS전선아시아,9/10/21,9/9/21,9/8/21,"숨은 진주, 3Q 호실적 & 신재생에너지","8,840","8,770",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"8,880","8,880","Hidden pearls, 3Q Human & Renewable Energy"


In [19]:
ex1 = translator.translate('NIM 방어, 비이자 회복세로 컨센서스 상회', src="ko", dest="en")
ex1.text

'NIM defense, non-interest recovery consensus'

In [24]:
merged_df.at[19280, 'en_report_title'] = ex1.text

In [25]:
merged_df.at[19280, 'en_report_title']

'NIM defense, non-interest recovery consensus'

In [23]:
ex2 = translator.translate('3Q 크게 상회. 분기 이익 1조원으로 레벨 업', src="ko", dest="en")
ex2.text

'3Q Large Open.Level up to quarterly profits'

In [26]:
merged_df.at[19323, 'en_report_title'] = ex2.text  #33032

In [27]:
ex3 = translator.translate('MLCC 이익 사이클', src="ko", dest="en")
ex3.text

'MLCC profit cycle'

In [28]:
merged_df.at[19366, 'en_report_title'] = ex3.text  #19366

In [30]:
merged_df.at[33032, 'report_title']

'현금 수지 개선 필요'

In [31]:
eng_title[33032]

'Cash resin needs to improve'

In [32]:
merged_df.at[33032, 'en_report_title'] = eng_title[33032]

In [43]:
merged_df.at[33032, 'report_title']

'현금 수지 개선 필요'

In [44]:
merged_df.at[33032, 'en_report_title']

'Cash resin needs to improve'

In [45]:
merged_df.to_pickle(PROJ_HOME+'/data/merged_translated_new.pkl')

In [64]:
translated_df = pd.read_pickle(PROJ_HOME+'/data/merged_translated_new.pkl')

# Copy translation from old file to new

In [65]:
merged_df[['report_title']]

,report_title
0,4분기 실적은 기대수준을 소폭 하회할 전망
1,폴리실리콘 원가개선 필요한 시점
2,"견조한 고무사업, 그러나 부진한 페놀유도체"
3,20년 연초에 꾸는 꿈
4,4분기 ESS 비용반영을 긍정적으로 보자
...,...
39795,호실적과 편안한 2022년
39796,공자위의 조속한 매각 결단을 환영
39797,저가 매수 유효
39798,"숨은 진주, 3Q 호실적 & 신재생에너지"


In [66]:
translated_df[['report_title']]

,report_title
0,4분기 실적은 기대수준을 소폭 하회할 전망
1,폴리실리콘 원가개선 필요한 시점
2,"견조한 고무사업, 그러나 부진한 페놀유도체"
3,20년 연초에 꾸는 꿈
4,4분기 ESS 비용반영을 긍정적으로 보자
...,...
39795,호실적과 편안한 2022년
39796,공자위의 조속한 매각 결단을 환영
39797,저가 매수 유효
39798,"숨은 진주, 3Q 호실적 & 신재생에너지"


In [67]:
for k, r in merged_df[['report_title']].iterrows():
    if r['report_title'] != translated_df.at[k, 'report_title']:
        print(r)
        break

In [68]:
merged_df.columns

Index(['author', 'source', 'ticker_symbol', 'company', 'report_date', '발행 전일',
       '발행 전전일', 'report_title', 'opening', 'closing', '고가', '전일종가', '전전일종가',
       'recommendation', '고가1', '고가2', '고가3', '고가4', '고가5', '고가6', '고가7',
       '고가8', '고가9', '고가10', '고가11', '고가12', '고가13', '고가14', '고가15', '고가16',
       '고가17', '고가18', '고가19', '고가20', '고가21', '고가22', '고가23', '고가24', '고가25',
       '고가26', '고가27', '고가28', '고가29', '고가30', '고가31', '고가32', '고가33', '고가34',
       '고가35', '고가36', '고가37', '고가38', '고가39', '고가40', '고가41', '고가42', '고가43',
       '고가44', '고가45', '고가46', '고가47', '고가48', '고가49', '고가50', 'H25', 'H50',
       'en_report_title'],
      dtype='object')

In [69]:
merged_df['en_report_title'] = translated_df['en_report_title']

In [70]:
merged_df[130:140]

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title
130,이동헌,대신,A047810,한국항공우주,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,납품 증가로 회복세 기대,33850,33900,...,25500.0,25000.0,25150.0,24100.0,21550.0,22050.0,20400.0,34700,34700,Expected to recover due to increased delivery
131,이동헌,대신,A012450,한화에어로스페이스,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,일회성 비용으로 숨고르기,35000,35150,...,27700.0,27200.0,26600.0,25400.0,22600.0,22900.0,21150.0,36450,36450,Hiding at one-off costs
132,이동헌,대신,A079550,LIG넥스원,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,방향은 맞지만,32900,33600,...,26200.0,25600.0,25400.0,24650.0,21800.0,24900.0,21200.0,35350,35350,The direction is right
133,이동헌,대신,NaN,현대두산인프라코어,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,낮은 기저로 무난한 실적 기대,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Low bases
134,이동헌,대신,A267270,현대건설기계,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,실적정체 지속,30400,29300,...,20100.0,19500.0,19650.0,18750.0,16000.0,15950.0,14700.0,31250,31250,Performance stagnation
135,이동헌,대신,A241560,두산밥캣,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,잠시 쉬어갈 뿐,33950,33550,...,28100.0,28250.0,27200.0,25250.0,23300.0,22400.0,20950.0,34600,34600,I'll take a break for a while
136,이동헌,대신,A034020,두산중공업,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,무난한 실적 기대,5208,5098,...,4542.0,4304.0,4281.0,3431.0,2866.0,2935.0,2880.0,5714,5714,Forty performance
137,이동헌,대신,A010120,LS ELECTRIC,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,융합사업부 실적개선 기대,51800,52200,...,44550.0,44400.0,43450.0,41250.0,36850.0,37400.0,33050.0,55100,55100,Improvement of Fusion Division
138,이동헌,대신,A267260,현대일렉트릭,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,끝나가는 적자 기조,11050,10900,...,9000.0,8750.0,8790.0,8150.0,6980.0,7080.0,6790.0,13350,13350,Finishing deficit
139,이동헌,대신,A298040,효성중공업,2020-01-06 00:00:00,2020-01-03 00:00:00,2020-01-02 00:00:00,중공업 부문 흑자전환 기대,27050,26100,...,18750.0,17800.0,18450.0,17000.0,14600.0,13650.0,12750.0,27050,27050,Heavy Industry Section


# data clean up

In [71]:
merged_df.reset_index().at[26949, 'source']

nan

In [72]:
merged_df.at[133, 'ticker_symbol']

nan

In [73]:
merged_df['ticker_symbol'] = merged_df['ticker_symbol'].fillna('')

In [74]:
for k, v in dict(merged_df['source'].fillna('')).items():
    if v.strip() == '':
        print(k, v)

26949 


In [75]:
# source nan to 리서치알음
merged_df.at[26949, 'source'] = '리서치알음'

In [76]:
merged_df[26940:26960]

,author,source,ticker_symbol,company,report_date,발행 전일,발행 전전일,report_title,opening,closing,...,고가44,고가45,고가46,고가47,고가48,고가49,고가50,H25,H50,en_report_title
26940,이문종,신한,A095660,네오위즈,2/10/21,2/9/21,2/8/21,"PER 8.7배, 상반기 신작 9종 출시",24900,24650,...,25100.0,25050.0,25850.0,25800.0,25500.0,24850.0,24500.0,"26,200","26,200","PER 8.7x, 9 new new work in the first half"
26941,김규리,신한,A228670,레이,2/10/21,2/9/21,2/8/21,4Q20 Review: 좋은 뉴스가 끊이질 않는다,60600,61700,...,58100.0,58600.0,58100.0,60400.0,59600.0,57200.0,55700.0,"62,400","62,400",4Q20 Review: Good news will not break
26942,성준원,신한,A114090,GKL,2/10/21,2/9/21,2/8/21,영업 중단 효과,15550,15450,...,15850.0,15700.0,15750.0,16000.0,15800.0,15700.0,15550.0,"17,500","17,500",Sales of business suspension
26943,한승재,DB금융,A011780,금호석유,2/10/21,2/9/21,2/8/21,천장이 뚫렸다,267000,268000,...,259000.0,255500.0,268000.0,271500.0,282000.0,292500.0,286500.0,"280,000","292,500",The ceiling was pierced.
26944,이문종,신한,A078340,컴투스,2/10/21,2/9/21,2/8/21,부진한 실적은 이미 반영,160000,156600,...,176000.0,178600.0,173000.0,170800.0,168300.0,172000.0,171800.0,"160,500","179,100",The sluggish performance is already reflected
26945,이문종,신한,A035720,카카오,2/10/21,2/9/21,2/8/21,상상 그 이상의 플랫폼 파워,466000,489500,...,132500.0,120500.0,122000.0,121000.0,119500.0,119500.0,118500.0,"519,000","561,000",Imagine platform power
26946,오강호,신한,A131290,티에스이,2/10/21,2/9/21,2/8/21,반등을 그리자,55200,54900,...,60100.0,59900.0,63300.0,63400.0,61700.0,61100.0,60700.0,"62,000","63,600",Half on the rebound
26947,김진구,KTB,A035720,카카오,2/10/21,2/9/21,2/8/21,글로벌 사업 강화 전략이 필요한 시기,466000,489500,...,132500.0,120500.0,122000.0,121000.0,119500.0,119500.0,118500.0,"519,000","561,000",When you need a global business enhancement st...
26948,박한샘,SK,A006650,대한유화,2/10/21,2/9/21,2/8/21,UHMWPE로 SPECIALTY 탈바꿈,323500,373500,...,308500.0,313000.0,313500.0,321500.0,322000.0,337000.0,333000.0,"405,500","405,500",SPECIALTY to UHMWPE
26949,리서치알음,리서치알음,A094480,갤럭시아머니트리,2/15/21,2/10/21,2/9/21,"힙한 재료 다 있네, 비트코인, 지역화폐, 쿠팡까지",4800,4690,...,7590.0,7510.0,7790.0,7900.0,7900.0,7500.0,7270.0,"7,950","8,890","There are hiped materials, bit coin, local cur..."


In [77]:
for k, v in dict(merged_df['report_title'].fillna('')).items():
    if v == '':
        print(k, v)

In [78]:
public_merged_df = merged_df.loc[merged_df['ticker_symbol'] != ''].copy()

In [79]:
public_merged_df.to_pickle(data_dir/'pub_traded_only.pkl')